In [4]:
r"""
This document seeks to implement polynomial multiplcation using the NTT (number theoretic transform) as 

NTT^{-1}(NTT(f(x)) \circ NTT(g(x)))

so we are just doing the forward NTT on each polynomial, performing the convolution, then taking the inverse.
""";

In [7]:
# Finite field setup
p = 17  # Prime modulus
n = 8   # Size of the transform (must divide p-1)
g = 3   # Primitive root modulo p

# Compute n-th root of unity
omega = pow(g, (p - 1) // n, p)

# Polynomial coefficients as integers modulo p
A = [1, 2, 3, 4, 0, 0, 0, 0]

# Modular arithmetic functions
def mod_add(a, b, p):
    return (a + b) % p

def mod_mul(a, b, p):
    return (a * b) % p

def mod_exp(base, exp, p):
    return pow(base, exp, p)

def mod_inv(a, p):
    return pow(a, -1, p)

# Forward transform using NTT
def ntt(a, omega, n, p):
    result = []
    for k in range(n):
        value = 0
        for j in range(n):
            value = mod_add(value, mod_mul(a[j], mod_exp(omega, j * k, p), p), p)
        result.append(value)
    return result

# Inverse transform using INTT
def intt(a, omega, n, p):
    omega_inv = mod_inv(omega, p)
    n_inv = mod_inv(n, p)
    result = []
    for k in range(n):
        value = 0
        for j in range(n):
            value = mod_add(value, mod_mul(a[j], mod_exp(omega_inv, j * k, p), p), p)
        result.append(mod_mul(value, n_inv, p))
    return result

# Perform forward and inverse transforms
A_ntt = ntt(A, omega, n, p)
A_recovered = intt(A_ntt, omega, n, p)

# Output results
print("Original:", A)
print("NTT:", A_ntt)
print("Recovered:", A_recovered)


9
Original: [1, 2, 3, 4, 0, 0, 0, 0]
NTT: [10, 16, 6, 11, 15, 13, 7, 15]
Recovered: [1, 2, 3, 4, 0, 0, 0, 0]


In [6]:
from sympy.discrete.transforms import ntt
from sympy import primerange

# Example parameters
seq = [1, 2, 3, 4, 0, 0, 0, 0]  # Input sequence (polynomial coefficients)
prime = 17  # A prime number

# Perform the Number Theoretic Transform
result = ntt(seq, prime)

print("Original sequence:", seq)
print("NTT result:", result)

Original sequence: [1, 2, 3, 4, 0, 0, 0, 0]
NTT result: [10, 16, 6, 11, 15, 13, 7, 15]


In [15]:
import galois
import numpy as np

# Define a prime field GF(p)
p = 17  # Choose a prime modulus
galois_field = galois.GF(p)

# Define input sequences over GF(p) as polynomial coefficients
a = galois_field([1, 2, 3, 4])  # Polynomial A
b = galois_field([5, 6, 7, 8])  # Polynomial B

# Pad polynomials to the length of the NTT transform (next power of 2)
n = 8  # Length of the NTT transform (could also be a power of 2, like 8)
a_padded = np.pad(a, (0, n - len(a)), mode='constant', constant_values=0)
b_padded = np.pad(b, (0, n - len(b)), mode='constant', constant_values=0)

# Perform the Number Theoretic Transform (NTT)
A = galois.ntt(a_padded, modulus=p)
B = galois.ntt(b_padded, modulus=p)

# Element-wise multiplication in the transformed domain
C = A * B

# Inverse NTT to retrieve the result in the original domain
c_ntt = galois.intt(C, modulus=p)

# Direct polynomial multiplication using numpy's polymul and modulo p
c_direct = np.polymul(a, b) % p

# Output the results
print("Polynomial A (coefficients):", a)
print("Polynomial B (coefficients):", b)
print("Padded Polynomial A:", a_padded)
print("Padded Polynomial B:", b_padded)
print("NTT of A:", A)
print("NTT of B:", B)
print("Pointwise Product in NTT Domain:", C)
print("Resultant Polynomial (from NTT):", c_ntt)
print("Direct Polynomial Multiplication (mod p):", c_direct)


Polynomial A (coefficients): [1 2 3 4]
Polynomial B (coefficients): [5 6 7 8]
Padded Polynomial A: [1 2 3 4 0 0 0 0]
Padded Polynomial B: [5 6 7 8 0 0 0 0]
NTT of A: [10 16  6 11 15 13  7 15]
NTT of B: [ 9 15  6  8 15  7  7  7]
Pointwise Product in NTT Domain: [ 5  2  2  3  4  6 15  3]
Resultant Polynomial (from NTT): [ 5 16  0  9 10  1 15  0]
Direct Polynomial Multiplication (mod p): [ 5 16  0  9 10  1 15]
